In [ ]:
!python -m spacy download en_core_web_md


In [6]:
import spacy
from transformers import AutoTokenizer, AutoModel
import torch

# Load models
nlp = spacy.load("en_core_web_md")
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# Define amplifiers with assigned strength
amplifiers = {"some": 1, "a few": 1, "several": 2, "many": 3, "most": 4, "all": 5, "every": 5, "always": 5}

def calculate_semantic_similarity(sentence1, sentence2):
    inputs1 = tokenizer(sentence1, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs2 = tokenizer(sentence2, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs1 = model(**inputs1)
        outputs2 = model(**inputs2)
    sentence_embedding1 = outputs1.last_hidden_state.mean(dim=1)
    sentence_embedding2 = outputs2.last_hidden_state.mean(dim=1)
    cosine_sim = torch.nn.functional.cosine_similarity(sentence_embedding1, sentence_embedding2)
    return cosine_sim.item()

def detect_amplifier_strength(sentence):
    doc = nlp(sentence)
    strength = 0
    for token in doc:
        for amp, val in amplifiers.items():
            if amp in token.text:
                strength = max(strength, val)
    return strength

def amplification_factor(sentence1, sentence2):
    semantic_similarity = calculate_semantic_similarity(sentence1, sentence2)
    # If the sentences are identical, set amplification factor to 0
    if semantic_similarity == 1:
        return 0

    strength1 = detect_amplifier_strength(sentence1)
    strength2 = detect_amplifier_strength(sentence2)

    # Calculate the amplification factor based on the presence and change in amplifier strength
    if strength2 > strength1:
        # Normalize the change in strength to a scale of 0 to 1
        change_in_strength = (strength2 - strength1) / max(amplifiers.values())
        # Factor in semantic similarity to ensure relevance
        factor = min(change_in_strength * semantic_similarity, 1.0)
    else:
        # No amplification or semantic relevance
        factor = semantic_similarity * 0.1  # Slight adjustment to account for some relevance

    return factor

# Example usage
sentence1 = "Some students passed the exam."
sentence2 = "Some students passed all the exams."
print("Amplification Factor:", amplification_factor(sentence1, sentence2))

sentence1 = "All students passed the exam."
sentence2 = "some students passed the exam."
print("Amplification Factor:", amplification_factor(sentence1, sentence2))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Amplification Factor: 0.9350095987319946
Amplification Factor: 0.1872041463851929
